## Preparation

In [19]:
include("multicaregiver.jl")
using JuMP

## II/ Solution

In [20]:
struct ValueIteration
    k_max # maximum number of iterations
end

### 1. Conditional Plan

#### 1.1 Struct

In [21]:
struct ConditionalPlan
    a # action to take at rootsubplan
    s # dictionary mapping observations to subplans
end

ConditionalPlan(a) = ConditionalPlan(a, Dict())

(π::ConditionalPlan)() = π.a
(π::ConditionalPlan)(o) = π.subplans[o]

#### 1.2 Utility

In [22]:
function lookahead(𝒫::CryingBaby, U, s, a)
    𝒮, 𝒪, T, O, R, γ = 𝒫.𝒮, joint(𝒫.𝒪), 𝒫.T, 𝒫.O, 𝒫.R, 𝒫.γ
    u′ = sum(T(s, a, s′)*sum(O(a, s′, o)*U(o, s′) for o in 𝒪) for s′ in 𝒮)
    return R(s, a) + γ*u′
end

function evaluate_plan(𝒫::CryingBaby, π, s)
    a = Tuple(πi() for πi in π)
    U(o, s′) = evaluate_plan(𝒫, [πi(oi) for (πi,oi) in zip(π, o)], s′)
    return isempty(first(π).subplans) ? 𝒫.R(s, a) : lookahead(𝒫, U, s, a)
end

function utility(𝒫::CryingBaby, b, π)
    u = [evaluate_plan(𝒫, π, s) for s in 𝒫.𝒮]
    return sum(bs*us for (bs,us) in zip(b, u))
end

utility (generic function with 4 methods)

#### 1.3 Alpha vector

In [23]:
function alphavector(𝒫::CryingBaby, π::ConditionalPlan)
    return[evaluate_plan(𝒫, π, s) for s in 𝒫.𝒮]
end

struct AlphaVectorPolicy
    𝒫 # POMDP problem
    Γ # alpha vectors
    a # actions associated with alpha vectors
end

function utility(π::AlphaVectorPolicy, b)
    return maximum(α⋅b for α in π.Γ)
end

function(π::AlphaVectorPolicy)(b)
    i = argmax([α⋅b for α in π.Γ])
    return π.a[i]
end

#### 1.4 Pruning

In [24]:
function lookahead(𝒫::CryingBaby, U, b::Vector, a)
    𝒮, 𝒪, T, O, R, γ = 𝒫.𝒮, 𝒫.𝒪, 𝒫.T, 𝒫.O, 𝒫.R, 𝒫.γ
    r = sum(R(s, a)*b[i] for (i, s) in enumerate(𝒮))
    Posa(o, s, a) = sum(O(a, s′, o)*T(s, a, s′) for s′ in 𝒮) 
    Poba(o, b, a) = sum(b[i]*Posa(o, s, a) for (i, s) in enumerate(𝒮))
    return r + γ*sum(Poba(o, b, a)*U(update(b, 𝒫, a, o)) for o in 𝒪)
end

function greedy(𝒫::CryingBaby, U, b::Vector)
    u, a = findmax(a->lookahead(𝒫, U, b, a), 𝒫.𝒜)
    return (a = a, u = u)
end

struct LookaheadAlphaVectorPolicy
    𝒫 # POMDP problem
    Γ # alpha vectors
end

function utility(π::LookaheadAlphaVectorPolicy, b)
    return maximum(α⋅b for α in π.Γ)
end 

function greedy(π, b)
    U(b) = utility(π, b)
    return greedy(π.𝒫, U, b)
end

(π::LookaheadAlphaVectorPolicy)(b) = greedy(π, b).a

In [25]:
function find_maximal_belief(α, Γ)
    m = length(α)
    if isempty(Γ)
        return fill(1/m, m) # arbitrary belief
    end
    model = Model(GLPK.Optimizer)
    @variable(model, δ)
    @variable(model, b[i=1:m] ≥ 0)
    @constraint(model, sum(b)==1.0)
    for a in Γ
        @constraint(model, (α-a)⋅b ≥ δ)
    end
    @objective(model, Max, δ)
    optimize!(model)
    return value(δ) > 0 ? value.(b) : nothing
end

find_maximal_belief (generic function with 1 method)

In [26]:
function find_dominating(Γ)
    n = length(Γ)
    candidates, dominating = trues(n), falses(n)
    while any(candidates)
        i = findfirst(candidates)
        b = find_maximal_belief(Γ[i],Γ[dominating])
        if b === nothing 
            candidates[i] = false
        else
            k = argmax([candidates[j] ? b⋅Γ[j] : -Inf for j in 1:n])
            candidates[k], dominating[k] = false, true 
        end
    end
    return dominating
end

function prune(plans, Γ)
    d = find_dominating(Γ)
    return (plans[d],Γ[d])
end

prune (generic function with 1 method)

#### 1.5 Value Iteration

In [27]:
function value_iteration(𝒫::CryingBaby, k_max)
    𝒮, 𝒜, R = 𝒫.𝒮, 𝒫.𝒜, 𝒫.R
    plans = [ConditionalPlan(a) for a in 𝒜]
    Γ = [[R(s, a) for s in 𝒮] for a in 𝒜]
    plans, Γ = prune(plans, Γ) 
    for k in 2:k_max
        plans, Γ = expand(plans, Γ, 𝒫)
        plans, Γ = prune(plans, Γ)
    end
    return (plans, Γ)
end

function solve(M::ValueIteration, 𝒫::CryingBaby)
    plans, Γ = value_iteration(𝒫, M.k_max)
    return LookaheadAlphaVectorPolicy(𝒫, Γ)
end

solve (generic function with 2 methods)

In [32]:
valueIteration = ValueIteration(10)
Baby = CryingBaby()

solve(valueIteration, Baby)

ErrorException: type CryingBaby has no field 𝒮